In [ ]:
import numpy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPooling1D, Conv1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
from os import listdir
from os.path import isfile, join
files = [f for f in listdir('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Data/Data/') if isfile(join('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Data/Data/', f))]

#Change path to your own drive
df_reviews = pd.DataFrame( columns=['review_text','review_summary','is_spoiler'])
for file in files:
  print("Appending: "+ file)
  df_temp = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Data/Data/' + file, lines = False)
  #df_temp = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Data/' + file, lines = False)
  #df_temp = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Clean/' + file, lines = False)
  df_reviews = df_reviews.append(df_temp.copy())



Note: Be sure to change the file path to Data_simplified_X(100) & Data_simplified_y(100) to your own path

In [ ]:
df_reviews.head()

,review_text,review_summary,is_spoiler
400000,"[go, watch, movie, largely, meryl, streep, ope...","[watch, blue, need, motivation, shop]",False
400001,"[movie, true, look, working, fashion, industry...","[meryl, streep, brilliant]",False
400002,"[anne, hathaway, play, intelligent, hard, work...","[delightful, comedy, like, even, bad, guy]",False
400003,"[actually, surprisingly, good, watch, thought,...","[surprising, good, realistic, portrayal, strug...",False
400004,"[absolutely, incredible, see, movie, four, tim...","[100, pure, entertainment]",False


In [ ]:
df_X = df_reviews[['review_text']].copy()
df_Y = df_reviews['is_spoiler'].copy()

In [ ]:
# Learn how to split test and training data from a whole
from sklearn.model_selection import train_test_split

# (random_state): we use a fixed random seed so we get the same results every time.
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y.T, test_size=0.2, random_state=0)

print ("Number of training instances: ", len(X_train), "\nNumber of test instances: ", len(X_test))

Number of training instances:  400730 
Number of test instances:  100183


In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train_encoded = lb.fit_transform(y_train)
y_test_encoded = lb.fit_transform(y_test)

In [ ]:
# sample tokenizer
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train['review_text']))
list_tokenized_train = tokenizer.texts_to_sequences(X_train['review_text'])
list_tokenized_test = tokenizer.texts_to_sequences(X_train['review_text'])
vocab_size = len(tokenizer.word_index) + 1
maxlen = 40
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)


In [ ]:
# setting parameter for tokenizer and padding
MAX_NB_WORDS = 30000
MAX_SEQUENCE_LENGTH = 200
############################################


if MAX_NB_WORDS == 0:
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(list(X_train['review_text']))
  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))
  MAX_NB_WORDS = len(word_index) + 1
else:
  tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
  tokenizer.fit_on_texts(list(X_train['review_text']))
  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))
  MAX_NB_WORDS = len(word_index) + 1

X_train_tok =  tokenizer.texts_to_sequences(X_train['review_text'])
X_test_tok =  tokenizer.texts_to_sequences(X_test['review_text'])



if MAX_SEQUENCE_LENGTH == 0:
  for text in X_train_tok:
      # find max_sequence_len for padding
      txt_to_seq_len = len(text)
      if txt_to_seq_len > MAX_SEQUENCE_LENGTH:
          MAX_SEQUENCE_LENGTH = txt_to_seq_len


# # truncate and pad input sequences
X_train_pad = sequence.pad_sequences(X_train_tok, maxlen=MAX_SEQUENCE_LENGTH)
X_test_pad = sequence.pad_sequences(X_test_tok, maxlen=MAX_SEQUENCE_LENGTH)

Found 825599 unique tokens.


In [ ]:
from gensim.models.keyedvectors import KeyedVectors
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
print("loading word2vec model…")
# word2vec_model = KeyedVectors.load_word2vec_format('./custom_review_100d.txt', binary=True)
word2vec_model = KeyedVectors.load_word2vec_format('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/w2v_model/custom_review_100d.txt')
def getVector(str):
    if str in word2vec_model:
        return word2vec_model[str]
    else:
        return None;


def isInModel(str):
    return str in word2vec_model

2021-11-06 07:41:55,935 : INFO : loading projection weights from /drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/w2v_model/custom_review_100d.txt


loading word2vec model…


2021-11-06 07:42:09,863 : INFO : loaded (92917, 100) matrix from /drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/w2v_model/custom_review_100d.txt


In [ ]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((MAX_NB_WORDS, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
  embedding_vector = getVector(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
# create the model
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(LSTM(60, return_sequences=True))
#model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train_pad, y_train_encoded, epochs=3, batch_size=64, validation_split=0.1, callbacks = callbacks_list)



#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
#model.add(MaxPooling1D(pool_size=2))


#model.fit(X_train, y_train, epochs=5, batch_size=256,verbose = 1,callbacks = callbacks_list,validation_data=(X_cv,y_cv))

ValueError: ignored

In [ ]:
accr = model.evaluate(X_train_pad,y_train_encoded)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
accr = model.evaluate(X_test_pad,y_test_encoded)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
# Final evaluation of the model
scores = model.evaluate(X_test_pad, y_test_encoded, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))